In [10]:
import pandas as pd

In [11]:
movies = pd.read_csv('movies_metadata.csv')

C:\Users\22575\AppData\Local\Temp\ipykernel_39240\2992503378.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('movies_metadata.csv')


In [13]:
len(movies)

45466

In [12]:
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
import string
class Geturl:
   
    def __init__(self, movies_file='movies_metadata.csv'):
        self.movies = pd.read_csv(movies_file)
        self.url = pd.DataFrame(columns=['title','imdb_id','url'])
        self.other_movies = pd.DataFrame(columns=['title','imdb_id'])

    def replace_punctuation_with_space(self,s):
        
        translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
        # 使用映射表替换标点符号
        return s.translate(translator)

    def is_letters_and_spaces(self,s):
        # 遍历字符串中的每个字符
        for char in s:
            # 检查字符是否是字母\数字或空格
            if not (char.isalnum() or char.isspace()):
                return False
        return True

    def get_urls(self):
        for  i in range(len(self.movies)):
            title = str(self.movies.iloc[i]['title']).strip()
            title_1 = title.replace('×', ' ').replace('·', ' ').replace('–', ' ').replace("…", ' ').replace("™", ' ').replace("’", '')
            title_s = self.replace_punctuation_with_space(title_1)
            if self.is_letters_and_spaces(title_s):
                title_r = title.replace(' ', '_').lower()
                imdb_id = self.movies.iloc[i]['imdb_id']
                url = f"https://www.rottentomatoes.com/m/{title_r}/reviews"
                #url_u = f"https://www.rottentomatoes.com/m/{title_r}/reviews?type=user"
                self.url.loc[i] = [title, imdb_id, url]
            else:
                imdb_id = self.movies.iloc[i]['imdb_id']
                self.other_movies.loc[i] = [title, imdb_id]

            

        return self.url , self.other_movies



In [85]:
urls ,other_movies = Geturl().get_urls()
urls.head(2)

C:\Users\22575\AppData\Local\Temp\ipykernel_18476\296017268.py:5: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  self.movies = pd.read_csv(movies_file)


,title,imdb_id,url
0,Toy Story,tt0114709,https://www.rottentomatoes.com/m/toy_story/rev...
1,Jumanji,tt0113497,https://www.rottentomatoes.com/m/jumanji/reviews


In [86]:
len(other_movies)

23

In [87]:
len(urls)

45443

In [88]:
other_movies.to_csv('imdb_reviews_other_movies.csv', index=False)

In [105]:
for i in range(38):
    file_name = f'imdb_reviews_urls_part_{i+1}.csv'
    urls_part = urls[i*1200:(i+1)*1200]
    urls_part.to_csv(file_name, index=False)

In [ ]:
ot-button-order-2

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from bs4 import BeautifulSoup

class IMDBReviewScraper:
    def __init__(self, urls_file):
        self.file_name = urls_file
        self.service = ChromeService()
        options = webdriver.ChromeOptions()
        self.driver = webdriver.Chrome(service=self.service, options=options)
        self.urls = pd.read_csv(urls_file)
        self.reviews_data = pd.DataFrame(columns=['title','imdb_id', 'reviews_c','reviews_u'])
        self.wrong_urls = pd.DataFrame(columns=['title','imdb_id','url'])
    # def scrape_review_page(self, page_source):
    #     soup = BeautifulSoup(page_source)
    #     reviews = soup.find_all(class_='review-text')
    # #     review_texts = [review.get_text().strip() for review in reviews]
    #     return review_texts
    def scrape_reviews_c(self):
       
        try :
            WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, "review-text")))
            
            reviews = self.driver.find_elements(By.CLASS_NAME, "review-text")
            review_texts = [review.text for review in reviews]
        except:
            review_texts = []
        return review_texts
    def scrape_reviews_u(self):
       
        # WebDriverWait(self.driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "audience-reviews__review js-review-text")))
            
        # reviews = self.driver.find_elements(By.CLASS_NAME, "audience-reviews__review js-review-text")
        # 用点号连接多个类名，作为CSS选择器
        
        try:
            WebDriverWait(self.driver, 3).until(EC.presence_of_element_located((By.CLASS_NAME, 'review-text-container')))
            rows = self.driver.find_elements(By.CLASS_NAME,'review-text-container')
            review_texts = [row.find_element(By.CLASS_NAME,'audience-reviews__review').text for row in rows]
        except:
            review_texts = []
        #review_texts = [review.text for review in reviews]
        return review_texts

    def close(self):
        self.driver.quit()
    def run(self):
        count = 0
        total_urls = len(self.urls)
        for i in range(len(self.urls)):
            url = self.urls.iloc[i]['url']
            self.driver.get(url)
            title = self.urls.iloc[i]['title']
            imdb_id = self.urls.iloc[i]['imdb_id']
            print(f"Processing {self.file_name}----{i+1}/{total_urls}: {title}, url: {url}")
            try:
                WebDriverWait(self.driver, 2).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()
            except:
                pass
            try :
                WebDriverWait(self.driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "sidebar-title"))
                )
            except:
                self.wrong_urls.loc[i] = {'title': title, 'imdb_id': imdb_id, 'url': url}
                continue
            # Scrape critic reviews
            # print(f"Scraped critic reviews for {title}, url: {url}")
            # 使用XPath查找按钮元素
            button_list = self.driver.find_element(By.CLASS_NAME,"tabs-wrap")
            buttons = button_list.find_elements(By.TAG_NAME, "rt-button")
            button_c = buttons[1]
            
            button_c.click()
            critic_reviews = self.scrape_reviews_c()
            # print(critic_reviews)
            # print (f"Scraped user reviews for {title}, url: {url}")
            button_list = self.driver.find_element(By.CLASS_NAME,"tabs-wrap")
            buttons = button_list.find_elements(By.TAG_NAME, "rt-button")
            button_u = buttons[2]
            button_u.click()
            user_reviews = self.scrape_reviews_u()
            # print(user_reviews)
            if len(critic_reviews)+len(user_reviews) !=0:
                count +=1
                print (f"Successfully scraped {count} out of {i+1} movies.")
                self.reviews_data.loc[i] = [ title,  imdb_id, critic_reviews, user_reviews]
                
        self.close()
        return self.reviews_data, self.wrong_urls



In [103]:
scraper = IMDBReviewScraper('reviews_test.csv')

In [104]:
testdata ,wrong_urls= scraper.run()
testdata

Scraped critic reviews for Toy Story, url: https://www.rottentomatoes.com/m/toy_story/reviews
["There's a giddy, absurd charm to the story, in which the strange setting only enhances the comfortable familiarity of the narrative and characters.", 'The simple story is enlivened by the distinctive personalities of the toys, especially the irascible tater head, the supportive Bo Peep lamp (voiced by Annie Potts) and a neurotic plastic dinosaur', 'More fun than Christmas mornings and birthday presents and winning a prize and making a friend, more fun than parking for free and falling in love and flying to Paris, more fun than anything you can think of is Toy Story.', 'The film contains a series of superb comic set-pieces and a finale which tries to equal the train chase at the end of The Wrong Trousers.', 'Toy Story is exciting but sweet-tempered, the kind of movie that parents and younger children can enjoy together.', 'With songs from Randy Newman and a screenplay full of cheeky wit, Toy 

,title,imdb_id,reviews_c,reviews_u
0,Toy Story,tt0114709,"[There's a giddy, absurd charm to the story, i...",[Once in a while Disney is able to produce a m...
1,Jumanji,tt0113497,[This sequel has enough razzle-dazzle and laug...,[Jumanji: Welcome to the Jungle is a hilarious...


In [ ]:
for i in range(38):
    file_name = f'imdb_reviews_urls_part_{i+1}.csv'
    scraper = IMDBReviewScraper(urls_file=file_name)
    reviews_data, wrong_urls = scraper.run()
    reviews_data.to_csv(f'imdb_reviews_data_part_{i+1}.csv', index=False)
    wrong_urls.to_csv(f'imdb_reviews_wrong_urls_part_{i+1}.csv', index=False)

In [ ]:
import requests

def get_url_content(url):
    try:
        # 发送 GET 请求
        response = requests.get(url)
        
        # 检查请求是否成功（状态码 200 表示成功）
        response.raise_for_status()
        
        # 返回响应内容
        return True
    
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP 错误: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"连接错误: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"超时错误: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"其他请求错误: {err}")
    
    return False

# 使用示例
if __name__ == "__main__":
    url = "https://www.rottentomatoes.com/m/one_day_since_yesterday_peter_bogdanovich_and_the_lost_american_film/reviews?type=top_critics"  # 替换为你要获取的 URL
    content = get_url_content(url)
    
    if content:
        print(f"成功获取 {url} 的内容：")


成功获取 https://www.rottentomatoes.com/m/one_day_since_yesterday_peter_bogdanovich_and_the_lost_american_film/reviews?type=top_critics 的内容：
<!DOCTYPE html>
<html lang="en" dir="ltr" xmlns="http://www.w3.org/1999/xhtml" prefix="fb: http://www.facebook.com/2008/fbml og: http://opengraphprotocol.org/schema/">
    <head prefix="og: http://ogp.me/ns# flixstertomatoes: http://ogp.me/ns/apps/flixstertomatoes#">

        
            
                <!-- OneTrust Cookies Consent Notice start for rottentomatoes.com --> <script src="https://cdn.cookielaw.org/consent/01978557-1604-76a7-ad7c-18216757cf52/otSDKStub.js"  type="text/javascript" c


In [2]:
pip install rottentomatoes-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
import rottentomatoes as rt

In [9]:
movie = rt.Movie('top gun maverick')


IndexError: list index out of range

In [8]:
def test() -> None:
   


test()

IndexError: list index out of range